In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import transforms, models
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import os
import logging
import math
import random
import pickle
import argparse
import traceback
from torchsummary import summary

from dataset import PEDataset
from utils import Timer

## Hyperparameters

In [2]:
load_feature_extractor = 'new/r3d_18_chunk10_batch16_new_splits/epoch9.pth'
feature_dir = 'test/features/new/r3d_18_chunk10_batch16_epoch10_new_splits'
gpu = 0
batch_size = 1
chunk_size = 10

## Data Splits

In [3]:
with open('data/data_splits.pkl', 'rb') as f:
    data_splits = pickle.load(f)

test_paths = data_splits['test_paths']
print("test {}".format(len(test_paths)))

test 1000


## Feature Extraction

In [4]:
transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((192, 192)),
    ])

test_set_s1 = PEDataset(paths=test_paths, mode='test', chunk_size=chunk_size,
                        transform=transform_test)

print("step 1 test exams: {}".format(test_set_s1.get_num_exams()))
print("step 1 test chunks: {}".format(len(test_set_s1)))

test_loader_s1 = DataLoader(test_set_s1, batch_size=1, shuffle=False)
print("step 1 test loader iters: {}".format(len(test_loader_s1)))

100%|██████████| 1000/1000 [00:00<00:00, 2033.46it/s]

step 1 test exams: 1000
step 1 test chunks: 26836
step 1 test loader iters: 26836


In [5]:
if load_feature_extractor == '':
    model = models.video.r3d_18(pretrained=True)
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, 1)
else:
    model = models.video.r3d_18(pretrained=False)
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, 1)
    model.load_state_dict(torch.load("weights/"+load_feature_extractor))

feature_extractor = torch.nn.Sequential(*(list(model.children())[:-1]))

print('')
print("Use cuda " + str(gpu))
print('')
device = torch.device("cuda:"+str(gpu) if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
feature_extractor.to(device)
print("Feature Extractor:")
summary(feature_extractor, input_size=(3, 10, 192, 192), batch_size=batch_size, device=device)
print('')

test_size = len(test_set_s1)

timer = Timer()

feature_extractor.eval()

test_features = np.zeros((test_size, 512))
test_labels = np.zeros((test_size, ))

timer.start()

for batch, (inputs, labels) in enumerate(test_loader_s1):
    inputs = inputs.to(device)
    with torch.no_grad():
        feature = feature_extractor(inputs)
        feature = feature.view(inputs.size(0), -1)
        start= batch * batch_size
        end = start + inputs.size(0)
        test_features[start:end] = feature.cpu().numpy()
        test_labels[start:end] = labels.cpu().numpy()

    if batch % 100 == 99:
        print('{} iters | time: {:.2f}'.format(batch+1, timer.stop()))
        timer.start()

timer.stop()


Use cuda 0

Feature Extractor:
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1        [1, 64, 10, 96, 96]          28,224
       BatchNorm3d-2        [1, 64, 10, 96, 96]             128
              ReLU-3        [1, 64, 10, 96, 96]               0
      Conv3DSimple-4        [1, 64, 10, 96, 96]         110,592
       BatchNorm3d-5        [1, 64, 10, 96, 96]             128
              ReLU-6        [1, 64, 10, 96, 96]               0
      Conv3DSimple-7        [1, 64, 10, 96, 96]         110,592
       BatchNorm3d-8        [1, 64, 10, 96, 96]             128
              ReLU-9        [1, 64, 10, 96, 96]               0
       BasicBlock-10        [1, 64, 10, 96, 96]               0
     Conv3DSimple-11        [1, 64, 10, 96, 96]         110,592
      BatchNorm3d-12        [1, 64, 10, 96, 96]             128
             ReLU-13        [1, 64, 10, 96, 96]               0
     Co

14000 iters | time: 3.05
14100 iters | time: 3.38
14200 iters | time: 3.49
14300 iters | time: 2.76
14400 iters | time: 2.65
14500 iters | time: 2.97
14600 iters | time: 3.08
14700 iters | time: 2.81
14800 iters | time: 2.92
14900 iters | time: 3.00
15000 iters | time: 2.99
15100 iters | time: 2.77
15200 iters | time: 3.45
15300 iters | time: 3.00
15400 iters | time: 2.94
15500 iters | time: 2.94
15600 iters | time: 2.61
15700 iters | time: 2.83
15800 iters | time: 3.06
15900 iters | time: 3.16
16000 iters | time: 2.82
16100 iters | time: 2.87
16200 iters | time: 2.64
16300 iters | time: 2.81
16400 iters | time: 2.84
16500 iters | time: 3.15
16600 iters | time: 2.57
16700 iters | time: 2.64
16800 iters | time: 3.83
16900 iters | time: 2.52
17000 iters | time: 2.57
17100 iters | time: 2.81
17200 iters | time: 3.40
17300 iters | time: 2.92
17400 iters | time: 2.93
17500 iters | time: 2.94
17600 iters | time: 2.64
17700 iters | time: 2.66
17800 iters | time: 2.75
17900 iters | time: 2.66


1.1151161193847656

In [6]:
if not os.path.exists(feature_dir):
    os.makedirs(feature_dir)
np.save(feature_dir+'/test_features', test_features)
np.save(feature_dir+'/test_labels', test_labels)